<a href="https://colab.research.google.com/github/afortuny/SustainableFashionAI/blob/main/CircularityAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Analyzing product reviews to understand circularity?

We will leverage the data from https://www.trailrunningreview.com/ , a leading company in product analysis, and we will evalute each trail running shoe from SS22 based on that dimensions:

Circularity:

*   Durability: Is the product make to last?
*   Versatility: can the product be used for multiple conditions /situations?
*   Sustainable materials: is the product made with organic, recycable or vegan materials?

Desirability:

*   Function: Is the product build up appropiate for its purpose?
*   Innovation: Is the product disrupting the market in some sense?
*   Price: Is the product affordable?



In [38]:
# Store url
url = 'https://www.trailrunningreview.com/es/Adidas-Terrex-Agravic-Pro/REVIEW--2640.html'

In [39]:
# Import `requests`
import requests

# Make the request and check object type
r = requests.get(url)
type(r)

requests.models.Response

In [40]:
# Extract HTML from Response object and print
html = r.text
#print(html)

In [41]:
# Import BeautifulSoup from bs4
from bs4 import BeautifulSoup


# Create a BeautifulSoup object from the HTML
soup = BeautifulSoup(html, "html5lib")
type(soup)

bs4.BeautifulSoup

In [44]:
soup.findAll('Review entera')[:8]

[]

In [42]:
text = soup.get_text()

In [43]:
text

'\n\n\nAdidas Terrex Agravic Pro - TRAILRUNNINGReview.com\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n@font-face {\n  font-family: \'Source Sans Pro\';\n  font-style: italic;\n  font-weight: 400;\n  font-display: swap;\n  src: url(https://fonts.gstatic.com/s/sourcesanspro/v21/6xK1dSBYKcSV-LCoeQqfX1RYOo3qPa7g.ttf) format(\'truetype\');\n}\n@font-face {\n  font-family: \'Source Sans Pro\';\n  font-style: italic;\n  font-weight: 700;\n  font-display: swap;\n  src: url(https://fonts.gstatic.com/s/sourcesanspro/v21/6xKwdSBYKcSV-LCoeQqfX1RYOo3qPZZclRdr.ttf) format(\'truetype\');\n}\n@font-face {\n  font-family: \'Source Sans Pro\';\n  font-style: italic;\n  font-weight: 900;\n  font-display: swap;\n  src: url(https://fonts.gstatic.com/s/sourcesanspro/v21/6xKwdSBYKcSV-LCoeQqfX1RYOo3qPZZklxdr.ttf) format(\'truetype\');\n}\n@font-face {\n  font-family: \'Source Sans Pro\';\n  font-style: normal;\n  font-weight: 400;\n  font-display: swap;\n  src: url(https://fonts.gstat